In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GRU, Dense
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

C:\Users\Samuel\AppData\Local\conda\conda\envs\myclone\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv("./data.csv")

In [4]:
smiles = df.iloc[:,0]
targets = df.iloc[:,1:13]
smiles_train, smiles_test, targets_train, targets_test = train_test_split(smiles, targets, test_size=0.2)

In [6]:
batch_size = 32
tokenizer = Tokenizer(filters='', lower=False, char_level=True)
tokenizer.fit_on_texts(smiles.values)
one_hot_train = tokenizer.texts_to_sequences(smiles_train.values)
one_hot_test = tokenizer.texts_to_sequences(smiles_test.values)
one_hot_train = pad_sequences(one_hot_train, padding='post')
one_hot_test = pad_sequences(one_hot_test, padding='post')
model = Sequential()
model.add(Embedding(len(tokenizer.index_docs) + 1, 50))
model.add(Conv1D(filters=192, kernel_size=3, strides=1))
model.add(GRU(units=224, return_sequences=True))
model.add(GRU(units=384))
model.add(Dense(12, activation='softmax'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(one_hot_train, targets_train, epochs=100, validation_split=0.2)
score = model.evaluate(one_hot_test, targets_test)
print(score)
